# Working with the Olympics.csv file in Pandas

The olympics dataset (olympics.csv) was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table).

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals.

We first do some cleaning on it to organize them in proper named columns and rows

In [1]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


### Example Entry

First Country in the DataFrame and its details

In [2]:
def example_entry():
    # This function returns the row for Afghanistan, which is a Series object. 
    return df.iloc[0]

example_entry() 

# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Name: Afghanistan, dtype: object

### Which country has won the most gold medals in summer games?

In [3]:
df[df['Gold'] == max(df['Gold'])].index[0]

'United States'

### Country which had the biggest difference between their summer and winter gold medal counts?

In [4]:
dfc = df[['Gold','Gold.1']]
dfc['Diff'] = abs(dfc['Gold']-dfc['Gold.1'])
dfc[dfc['Diff'] == max(dfc['Diff'])].index[0]

<ipython-input-4-1513e96687dd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc['Diff'] = abs(dfc['Gold']-dfc['Gold.1'])


'United States'

### Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

[To include countries that have won at least 1 gold in both summer and winter]

In [5]:
new = df[['Gold','Gold.1','Gold.2']][(df['Gold']>0) & (df['Gold.1']>0)]
new['Ratio']= (abs(new['Gold']-new['Gold.1']))/new['Gold.2']
new[new['Ratio']== max(new['Ratio'])].index[0]

'Bulgaria'

### A function that creates a Series called "Points"
Points will be a weighted value where each gold medal (`Gold.2`) counts for 3 points, silver medals (`Silver.2`) for 2 points, and bronze medals (`Bronze.2`) for 1 point.

In [6]:
def Points():
    df['Points']=(3*df['Gold.2']) + (2*df['Silver.2']) + df['Bronze.2']
    return df['Points']

Points()

Afghanistan                           2
Algeria                              27
Argentina                           130
Armenia                              16
Australasia                          22
                                   ... 
Yugoslavia                          171
Independent Olympic Participants      4
Zambia                                3
Zimbabwe                             18
Mixed team                           38
Name: Points, Length: 146, dtype: int64